# Simulating loop extrusion throught CTCF sites and their features

In [24]:
import src 
import pandas as pd
import numpy as np
from scipy.special import softmax
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing.pool import Pool
import os

## Load ctcf binding sites and their features

In [25]:
ctcf_scores = pd.read_csv(src.interim_data_path / "ctcf_scores.tsv", sep="\t")
ctcf_scores = pd.concat((ctcf_scores, pd.DataFrame(data=src.quantile_normalization(ctcf_scores[['MotifScore', 'ChipSeqScore']].values), 
                                                   columns=['MotifScore_qnorm', 'ChipSeqScore_qnorm'])), axis=1)
ctcf_scores['AggregateScore_qnorm'] = ctcf_scores.MotifScore_qnorm + ctcf_scores.ChipSeqScore_qnorm
ctcf_scores.head()

,chr,start,end,orientation,ctcf_id,MotifScore,ChipSeqScore,rank_score_aggregate,MotifScore_qnorm,ChipSeqScore_qnorm,AggregateScore_qnorm
0,chr1,237593,237953,>,0,9.790746,12.639115,1.502743e+09,11.517411,11.156743,22.674153
1,chr1,521337,521697,>,1,10.259412,13.836791,1.680162e+09,13.291915,11.856638,25.148554
2,chr1,714087,714447,>,2,9.893988,9.326291,1.342770e+09,11.909662,9.128776,21.038438
3,chr1,805232,805362,>,3,14.365820,46.641218,3.446545e+09,37.311965,29.945322,67.257287
4,chr1,839966,840326,>,4,16.393501,60.468042,3.756117e+09,55.565129,37.427485,92.992614


## Get stopping probabilities out of features

In [26]:
def logistic(x, L=1, k=1, mu=0):
    return L / (1 + np.exp(-k*(x - mu)))

def prob_perfect_looping(data):
    return np.ones(data.shape[0])

def assign_probs(ctcf_scores, probs):
    return ctcf_scores.iloc[:, :5].assign(stopping_prob=probs)

#### Perfect looping

In [29]:
ctcfs_with_probs = assign_probs(ctcf_scores, prob_perfect_looping(ctcf_scores))
ctcfs_with_probs.head(5)

,chr,start,end,orientation,ctcf_id,stopping_prob
0,chr1,237593,237953,>,0,1.0
1,chr1,521337,521697,>,1,1.0
2,chr1,714087,714447,>,2,1.0
3,chr1,805232,805362,>,3,1.0
4,chr1,839966,840326,>,4,1.0


#### Motif Score

In [30]:
# ctcfs_with_probs = assign_probs(ctcf_scores, src.min_max_normalization(ctcf_scores.MotifScore.values))
# ctcfs_with_probs.head(5)

#### ChipSeq Score

In [31]:
# ctcfs_with_probs = assign_probs(ctcf_scores, min_max_normalization(ctcf_scores.ChipSeqScore.values))
# ctcfs_with_probs.head(5)

#### Rank score aggregate

In [32]:
# ctcfs_with_probs = assign_probs(ctcf_scores, src.min_max_normalization(ctcf_scores.rank_score_aggregate.values))
# ctcfs_with_probs.head(5)

## Do the simulation

In [33]:
gaps = pd.read_csv(src.external_data_path / "hg19_gaps.txt", sep="\t")
gaps.columns = ['chr', 'start', 'end', 'gap_type']
gaps = gaps.sort_values(src.coords)
gaps.head()

,chr,start,end,gap_type
1,chr1,0,10000,telomere
0,chr1,121535434,124535434,centromere
2,chr1,249240621,249250621,telomere
35,chr10,0,10000,telomere
33,chr10,39254935,42254935,centromere


In [34]:
random_seed = 42

LEFT_STOP_SYMBOL = '>'
BIDIRECTIONAL_STOP_SYMBOL = 'o'
RIGHT_STOP_SYMBOL = '<'


def get_loops(ctcfs_with_probs, random_seed=42):
    np.random.seed(random_seed)
    ctcfs_with_probs = ctcfs_with_probs.sort_values("ctcf_id")
    ids = ctcfs_with_probs.ctcf_id.values
    probs = ctcfs_with_probs.stopping_prob.values
    orients = ctcfs_with_probs.orientation.values
    
    loops = []
    middle_poss = np.arange(ids.shape[0] - 1)
    for mp in middle_poss:
        #left 
        lp = mp
        while ( np.random.uniform() > (0 if orients[lp] not in [LEFT_STOP_SYMBOL, BIDIRECTIONAL_STOP_SYMBOL] else 1)*probs[lp] ) and (lp >= 0):
            lp = lp - 1
        #right
        rp = mp + 1
        while ( np.random.uniform() > (0 if orients[rp] not in [RIGHT_STOP_SYMBOL, BIDIRECTIONAL_STOP_SYMBOL] else 1)*probs[rp] ) and (rp < middle_poss.shape[0]):
            rp = rp + 1

        if (lp >= 0) and (rp < middle_poss.shape[0]):
            loops.append({
                'left_id': ids[lp],
                'right_id': ids[rp],
                'count': 1
            })
    loops = pd.DataFrame.from_dict(loops)
    return loops

def merge_simulations(loops):
    result = loops.groupby(['left_id', 'right_id'])['count'].sum().reset_index()
    return result.sort_values(['left_id', 'right_id'])

def simulate_one_epoch(ctcfs_with_probs, random_seed=42):
    loops = []
    for chrom in sorted(ctcfs_with_probs.chr.unique()):
        chrom_data = ctcfs_with_probs[ctcfs_with_probs.chr == chrom]
        before_centromere = chrom_data[chrom_data.end < gaps[(gaps.chr == chrom) & (gaps.gap_type == 'centromere')].iloc[0].start]
        loops_before = get_loops(before_centromere, random_seed)
        loops.append(loops_before)
        after_centromere = chrom_data[chrom_data.start > gaps[(gaps.chr == chrom) & (gaps.gap_type == 'centromere')].iloc[0].end]
        loops_after = get_loops(after_centromere, random_seed)
        loops.append(loops_after)
    loops = merge_simulations(pd.concat(loops, axis=0, ignore_index=True))
    return loops

def simulate(ctcfs_with_probs, 
             epochs=10):
    with Pool(10) as pool:
        result = pool.starmap(simulate_one_epoch, [(ctcfs_with_probs, random_seed + i) for i in range(epochs)])
        result = merge_simulations(pd.concat(result, axis=0, ignore_index=True))
    return result

In [35]:
n_epochs = 100

In [36]:
simulation = simulate(ctcfs_with_probs, epochs=n_epochs)
print("# loops:", simulation.shape[0])
simulation.head()

# loops: 62418


,left_id,right_id,count
0,0,6,100
1,1,6,100
2,2,6,100
3,3,6,100
4,4,6,100


### Save the result

In [37]:
name = 'perfect'

In [38]:
os.makedirs(src.interim_data_path / "simulations", exist_ok=True)
simulation.to_csv(src.interim_data_path / "simulations" / 'sim_with_o_prob_{}_epochs_{}.tsv'.format(name, n_epochs), sep="\t", index=False)

In [39]:
simulation = pd.read_csv(src.interim_data_path / "simulations" / 'sim_with_o_prob_{}_epochs_{}.tsv'.format(name, n_epochs), sep="\t")
simulation.head()

,left_id,right_id,count
0,0,6,100
1,1,6,100
2,2,6,100
3,3,6,100
4,4,6,100


In [40]:
def __get_loop_polarity(loop):
    left_id, right_id = loop.left_id, loop.right_id
    between_orientations = ctcf_scores.loc[left_id+1:right_id-1].orientation.tolist()
    polarity = between_orientations.count("<") - between_orientations.count(">")
    return polarity

def __to_color(polarity):
    if polarity < 0:
        return "#0000FF" #blue
    elif polarity > 0:
        return "#FF0000" #red
    else:
        return "#000000" #black
    
def to_UCSC_interact(loops):
    return pd.DataFrame({
            "chrom": loops.left_chr,
            'chromStart': loops.left_start,
            'chromEnd': loops.right_end,
            'name': '.',
            'score': loops.score,
            'value': 1,
            'exp': '.',
            'color': loops.color,
            'sourceChrom': loops.left_chr,
            'sourceStart': loops.left_start,
            'sourceEnd': loops.left_end,
            'sourceName': loops.left_ctcf_id,
            'sourceStrand': '.',
            'targetChrom': loops.right_chr,
            'targetStart': loops.right_start,
            'targetEnd': loops.right_end,
            'targetName': loops.right_ctcf_id,
            'targetStrand': '.'
        })

In [44]:
simulation_annotated = simulation.copy()
simulation_annotated['polarity'] = simulation_annotated.apply(__get_loop_polarity, axis=1)

In [45]:
simulation_annotated = simulation_annotated\
    .merge(ctcf_scores.rename(columns= lambda x: "left_" + x), left_on="left_id", right_on="left_ctcf_id")\
    .merge(ctcf_scores.rename(columns= lambda x: "right_" + x), left_on='right_id', right_on='right_ctcf_id')\
    .assign(score = lambda x: (((x['count'] - x['count'].min() + 1) / (x['count'].max() - x['count'].min() + 1))*1000).astype(int))
simulation_annotated['color'] = simulation_annotated.polarity.map(__to_color)
simulation_annotated.head()

,left_id,right_id,count,polarity,left_chr,left_start,left_end,left_orientation,left_ctcf_id,left_MotifScore,...,right_orientation,right_ctcf_id,right_MotifScore,right_ChipSeqScore,right_rank_score_aggregate,right_MotifScore_qnorm,right_ChipSeqScore_qnorm,right_AggregateScore_qnorm,score,color
0,0,6,100,-5,chr1,237593,237953,>,0,9.790746,...,<,6,9.596212,5.783345,1.002850e+09,10.912164,6.763733,17.675897,1000,#0000FF
1,1,6,100,-4,chr1,521337,521697,>,1,10.259412,...,<,6,9.596212,5.783345,1.002850e+09,10.912164,6.763733,17.675897,1000,#0000FF
2,2,6,100,-3,chr1,714087,714447,>,2,9.893988,...,<,6,9.596212,5.783345,1.002850e+09,10.912164,6.763733,17.675897,1000,#0000FF
3,3,6,100,-2,chr1,805232,805362,>,3,14.365820,...,<,6,9.596212,5.783345,1.002850e+09,10.912164,6.763733,17.675897,1000,#0000FF
4,4,6,100,-1,chr1,839966,840326,>,4,16.393501,...,<,6,9.596212,5.783345,1.002850e+09,10.912164,6.763733,17.675897,1000,#0000FF


In [46]:
simulation_annotated_UCSC = to_UCSC_interact(simulation_annotated)
simulation_annotated_UCSC.head()

,chrom,chromStart,chromEnd,name,score,value,exp,color,sourceChrom,sourceStart,sourceEnd,sourceName,sourceStrand,targetChrom,targetStart,targetEnd,targetName,targetStrand
0,chr1,237593,856704,.,1000,1,.,#0000FF,chr1,237593,237953,0,.,chr1,856454,856704,6,.
1,chr1,521337,856704,.,1000,1,.,#0000FF,chr1,521337,521697,1,.,chr1,856454,856704,6,.
2,chr1,714087,856704,.,1000,1,.,#0000FF,chr1,714087,714447,2,.,chr1,856454,856704,6,.
3,chr1,805232,856704,.,1000,1,.,#0000FF,chr1,805232,805362,3,.,chr1,856454,856704,6,.
4,chr1,839966,856704,.,1000,1,.,#0000FF,chr1,839966,840326,4,.,chr1,856454,856704,6,.


In [47]:
simulation_annotated_UCSC.to_csv(src.interim_data_path / 'sim_with_o_prob_{}_epochs_{}.tsv'.format(name, n_epochs), sep="\t", index=False, header=False)